<<<FILE: .env.example >>>\n# AltTrade AI Bot (ProfitBeast Edition) .env.example
# Copy to .env and fill in real values for production. Default mode is sandbox.

ENV_MODE=sandbox               # sandbox | live
TELEGRAM_BOT_TOKEN=            # required
ADMIN_TELEGRAM_IDS=123456789   # comma-separated Telegram IDs with admin privileges
DB_PATH=/mnt/data/alttrade.db

# Exchange (CCXT) keys (live only)
CCXT_EXCHANGE=binance
CCXT_API_KEY=
CCXT_API_SECRET=
LIVE_MODE=false

# LLM providers
GROQ_API_KEY=                   # preferred LLM (free/your key)
OPENAI_API_KEY=                 # optional fallback; included for future use

# Payments (sandbox stubs unless configured)
STRIPE_API_KEY=
MPESA_CONSUMER_KEY=
MPESA_CONSUMER_SECRET=
BINANCE_PAY_KEY=

# Logger
LOG_JSON=false
LOG_LEVEL=INFO

# App config
TELEGRAM_POLLING=true
TRIAL_DAYS=7


<<<FILE: src/config.py >>>\nimport os
from pathlib import Path

ROOT = Path(__file__).parent.parent
ENV_MODE = os.getenv("ENV_MODE", "sandbox")
TELEGRAM_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN", "")
ADMIN_TELEGRAM_IDS = [int(x.strip()) for x in os.getenv("ADMIN_TELEGRAM_IDS","").split(",") if x.strip().isdigit()]
DB_PATH = os.getenv("DB_PATH", "/mnt/data/alttrade.db")
CCXT_EXCHANGE = os.getenv("CCXT_EXCHANGE", "binance")
LIVE_MODE = os.getenv("LIVE_MODE", "false").lower() in ("1","true","yes")
GROQ_API_KEY = os.getenv("GROQ_API_KEY", "")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")
TRIAL_DAYS = int(os.getenv("TRIAL_DAYS", "7"))

LOG_JSON = os.getenv("LOG_JSON","false").lower() in ("1","true","yes")
LOG_LEVEL = os.getenv("LOG_LEVEL","INFO")
TELEGRAM_POLLING = os.getenv("TELEGRAM_POLLING","true").lower() in ("1","true","yes")


<<<FILE: src/logger_setup.py >>>\nimport logging, sys, json
from .config import LOG_JSON, LOG_LEVEL

def get_logger(name=__name__):
    level = getattr(logging, LOG_LEVEL.upper(), logging.INFO)
    logger = logging.getLogger(name)
    if logger.handlers:
        return logger
    logger.setLevel(level)
    handler = logging.StreamHandler(sys.stdout)
    if LOG_JSON:
        formatter = JsonFormatter()
    else:
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

class JsonFormatter(logging.Formatter):
    def format(self, record):
        data = {
            "time": self.formatTime(record, self.datefmt),
            "name": record.name,
            "level": record.levelname,
            "message": record.getMessage(),
        }
        if record.exc_info:
            data["exc"] = self.formatException(record.exc_info)
        return json.dumps(data)

if __name__ == "__main__":
    log = get_logger("logger_setup_test")
    log.info("Logger self-test successful")


Repository manifest:

<<<FILE: src/trading/live_ccxt.py >>>\n"""Live CCXT wrapper. Reads CCXT_EXCHANGE, CCXT_API_KEY, CCXT_API_SECRET from env.
Provides fetch_balance and create_order with safety checks (only if LIVE_MODE==True).
"""
import os, ccxt, logging
from ..config import LIVE_MODE, CCXT_EXCHANGE
from ..logger_setup import get_logger

log = get_logger(__name__)

def get_live_exchange():
    if not LIVE_MODE:
        raise RuntimeError('LIVE_MODE is not enabled. Set LIVE_MODE=true in .env to enable live trading.')
    api_key = os.getenv('CCXT_API_KEY') or os.getenv('EXCHANGE_API_KEY')
    api_secret = os.getenv('CCXT_API_SECRET') or os.getenv('EXCHANGE_API_SECRET')
    if not api_key or not api_secret:
        raise RuntimeError('CCXT API keys not found in environment.')
    exchange_id = CCXT_EXCHANGE
    try:
        ex_class = getattr(ccxt, exchange_id)
    except Exception as e:
        # try lowercase
        ex_class = getattr(ccxt, exchange_id.lower())
    exchange = ex_class({
        'apiKey': api_key,
        'secret': api_secret,
        'enableRateLimit': True,
    })
    log.info('Initialized CCXT exchange: %s', exchange_id)
    return exchange

class LiveExchange:
    def __init__(self):
        self.exchange = get_live_exchange()
    def fetch_balance(self):
        return self.exchange.fetch_balance()
    def create_order(self, symbol, side, amount, price=None, order_type='limit'):
        if not LIVE_MODE:
            raise RuntimeError('Live orders disabled.')
        if order_type == 'market':
            order = self.exchange.create_market_order(symbol, side, amount)
        else:
            order = self.exchange.create_limit_order(symbol, side, amount, price)
        log.info('Placed live order: %s', order)
        return order

if __name__ == '__main__':
    try:
        ex = LiveExchange()
        print(ex.fetch_balance())
    except Exception as e:
        print('Live CCXT initialization error:', e)


<<<FILE: src/llm/groq_client.py >>>\n"""Groq LLM client wrapper. Uses GROQ_API_KEY from env to call Groq inference endpoints.
If GROQ_API_KEY is not present, falls back to deterministic template. Also supports OPENAI as fallback.
""" 
import os, requests, json
from ..config import GROQ_API_KEY, OPENAI_API_KEY
from ..logger_setup import get_logger
log = get_logger(__name__)

GROQ_API_URL = os.getenv('GROQ_API_URL', 'https://api.groq.ai/v1')  # placeholder

def _call_groq(prompt, max_tokens=256):
    if not GROQ_API_KEY:
        raise RuntimeError('GROQ_API_KEY not set')
    headers = {
        'Authorization': f'Bearer {GROQ_API_KEY}',
        'Content-Type': 'application/json'
    }
    payload = {
        'prompt': prompt,
        'max_tokens': max_tokens
    }
    # NOTE: This is a generic placeholder; adapt to actual Groq API shape.
    resp = requests.post(f'{GROQ_API_URL}/generate', headers=headers, json=payload, timeout=10)
    resp.raise_for_status()
    data = resp.json()
    # expecting {'text': '...'} or similar
    text = data.get('text') or data.get('output') or json.dumps(data)
    return text

def explain_signal(signal_type, context):
    """Return a human-readable explanation for an ML signal.
    If GROQ_API_KEY provided, call Groq; if not, deterministic fallback.
    """
    prompt = f"Explain {signal_type} alert with context: {json.dumps(context)}. Provide concise actionable steps and risk level."
    try:
        if GROQ_API_KEY:
            log.info('Calling Groq for LLM explanation.')
            return _call_groq(prompt)
        elif OPENAI_API_KEY:
            # Placeholder for OpenAI integration
            log.info('OPENAI_API_KEY present; OpenAI integration can be implemented here.')
        # Fallback deterministic explanation
        explanation = f"[LLM Explanation - {signal_type}] Observed context: {context}. Confidence: medium. Recommended action: review and monitor."
        return explanation
    except Exception as e:
        log.exception('LLM call failed, returning fallback explanation.')
        return f"[LLM-Fallback] Could not call LLM: {e}. Context: {context}"


<<<FILE: src/payments/stripe_live.py >>>\n"""Stripe live provider integration. Requires STRIPE_API_KEY env var.
Provides create_payment_intent and webhook verification functions.
"""
import os
from ..logger_setup import get_logger
log = get_logger(__name__)

try:
    import stripe
except Exception:
    stripe = None

def init_stripe():
    key = os.getenv('STRIPE_API_KEY') or os.getenv('STRIPE_SECRET_KEY')
    if not key:
        raise RuntimeError('STRIPE_API_KEY not set in env')
    if not stripe:
        raise RuntimeError('stripe library not installed')
    stripe.api_key = key
    return stripe

def create_payment_intent(amount_cents, currency='usd', metadata=None):
    s = init_stripe()
    intent = s.PaymentIntent.create(amount=amount_cents, currency=currency, metadata=metadata or {})
    log.info('Created Stripe PaymentIntent %s', intent.id)
    return intent

def verify_webhook(payload, sig_header, endpoint_secret):
    s = init_stripe()
    try:
        event = s.Webhook.construct_event(payload, sig_header, endpoint_secret)
        return event
    except Exception as e:
        log.exception('Stripe webhook verification failed.')
        raise


<<<FILE: src/web/admin_api.py >>>\n"""FastAPI admin dashboard for AltTrade. Provides admin endpoints and webhook handlers.
Admin access is protected by ADMIN_TELEGRAM_IDS and an ADMIN_API_KEY env var for web access.
""" 
import os, json
from fastapi import FastAPI, Request, HTTPException, Depends
from fastapi.responses import JSONResponse
from ..storage.db import init_db
from ..config import ADMIN_TELEGRAM_IDS
from ..logger_setup import get_logger
from ..logger_advanced import audit_event
from ..marketplace.market import list_items, approve_item
from ..agents.manager import start_agent, stop_agent, AGENTS

log = get_logger(__name__)
app = FastAPI()
conn = init_db()
ADMIN_API_KEY = os.getenv('ADMIN_API_KEY', '')

def check_admin(api_key: str = None):
    # allow admin via API key (for web) or via token param in headers
    if ADMIN_API_KEY and api_key == ADMIN_API_KEY:
        return True
    raise HTTPException(status_code=403, detail='Forbidden')

@app.get('/admin/users')
def get_users(api_key: str = None):
    check_admin(api_key)
    c = conn.cursor()
    c.execute('SELECT telegram_id, username, created_at, is_premium FROM users')
    rows = c.fetchall()
    return {'users': rows}

@app.post('/admin/marketplace/{item_id}/approve')
def approve_marketplace(item_id: int, api_key: str = None):
    check_admin(api_key)
    # perform approval
    approve_item('web_admin', item_id)
    audit_event('marketplace_approve', {'item_id': item_id}, telegram_id=None)
    return {'status':'approved', 'item_id': item_id}

@app.post('/admin/agent/start')
def api_start_agent(owner_id: int, symbol: str = 'BTC/USDT', interval: int = 5, api_key: str = None):
    check_admin(api_key)
    agent_id = start_agent(owner_id, symbol, interval)
    return {'agent_id': agent_id}

@app.post('/admin/agent/stop')
def api_stop_agent(agent_id: str, api_key: str = None):
    check_admin(api_key)
    ok = stop_agent(agent_id)
    return {'stopped': ok}

@app.post('/webhook/stripe')
async def stripe_webhook(request: Request):
    payload = await request.body()
    sig_header = request.headers.get('stripe-signature')
    # For now, just log and return success
    audit_event('stripe_webhook', {'headers': dict(request.headers)}, telegram_id=None)
    return JSONResponse({'received': True})

@app.post('/webhook/mpesa')
async def mpesa_webhook(request: Request):
    data = await request.json()
    audit_event('mpesa_webhook', data, telegram_id=None)
    return JSONResponse({'received': True})
